In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

MainProcess


In [3]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

(444, 200)

In [ ]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.seed = 42
model.use_pretrained_weights = False
model.n_epochs = 30
model.run_in_parallel = True
if fast_mode: model.n_epochs = 1
model.train(train_data, test_data)
if not fls.env=='kaggle':
    fls.dill_save(fls.model_dir+ "yolo.pickle", model)

CompletedProcess(args=['pip', 'uninstall', '-y', 'albumentations'], returncode=0)
Will process approximately 3123 slices for training


Processing training motors:   0%|          | 0/347 [00:00<?, ?it/s]

Clearing pytorch
Will process approximately 900 slices for validation


Processing validation motors:   0%|          | 0/100 [00:00<?, ?it/s]


Preprocessing Complete:
- Training data: 258 tomograms, 347 motors, 3118 slices
- Validation data: 100 tomograms, 100 motors, 900 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Created new YAML at d:/flagellar/temp/training.yaml
Using YAML file: d:/flagellar/temp/training.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: images/train
val: images/val


Starting YOLO training...
New https://pypi.org/project/ultralytics/8.3.104 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.yaml, data=d:/flagellar/temp/training.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=4, project=d:/flagellar/temp//yolo_


train: Scanning D:\flagellar\temp\yolo_dataset\labels\train... 3118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 

train: New cache created: D:\flagellar\temp\yolo_dataset\labels\train.cache



val: Scanning D:\flagellar\temp\yolo_dataset\labels\val... 900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 900/9

val: New cache created: D:\flagellar\temp\yolo_dataset\labels\val.cache
Plotting labels to d:\flagellar\temp\yolo_weights\motor_detector\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to d:\flagellar\temp\yolo_weights\motor_detector
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      6.31G      3.345      31.36      1.242         28        640:  29%|██▊       | 56/195 [00:19<00:36,  3

In [ ]:
if not fls.is_submission:
    importlib.reload(flg_yolo)    
    inferred_test_data = model.infer(test_data[:20])
    print(fls.score_competition_metric(inferred_test_data, test_data[:20]))

In [ ]:
inferred_test_data2 = model.infer(fls.load_all_test_data())
fls.write_submission_file(inferred_test_data2)

In [ ]:
inferred_test_data2